<a href="https://colab.research.google.com/github/vineet96/vertex-ai-class/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
PROJECT_ID = "vertexai-dev-387420"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
BUCKET_URI = f"gs://'classification'-{PROJECT_ID}-unique"  # @param {type:"string"}

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://classification-vertexai-dev-387420-unique/...


In [ ]:
import google.cloud.aiplatform as aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

Creating ImageDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating ImageDataset


Create ImageDataset backing LRO: projects/1052777806387/locations/us-central1/datasets/273186858060480512/operations/2872894347165040640


INFO:google.cloud.aiplatform.datasets.dataset:Create ImageDataset backing LRO: projects/1052777806387/locations/us-central1/datasets/273186858060480512/operations/2872894347165040640


ImageDataset created. Resource name: projects/1052777806387/locations/us-central1/datasets/273186858060480512


...

INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.ImageDataset('projects/1052777806387/locations/us-central1/datasets/273186858060480512')


Importing ImageDataset data: projects/1052777806387/locations/us-central1/datasets/273186858060480512


INFO:google.cloud.aiplatform.datasets.dataset:Importing ImageDataset data: projects/1052777806387/locations/us-central1/datasets/273186858060480512


Import ImageDataset data backing LRO: projects/1052777806387/locations/us-central1/datasets/273186858060480512/operations/8303391072843726848


INFO:google.cloud.aiplatform.datasets.dataset:Import ImageDataset data backing LRO: projects/1052777806387/locations/us-central1/datasets/273186858060480512/operations/8303391072843726848


In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers",
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)